<a href="https://colab.research.google.com/github/adoc2002/AdvDeepLearn/blob/main/ADL_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [27]:
# Set Hyperparameters

batch_size=32  # How many independent sequences we process in parallel
block_size=8 # What is the maximum context length for prediction
max_iters=5000
eval_interval=500
learning_rate= 3e-4


device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iterations=200
n_embd= 384 # Number of embedding dimensions
n_head= 6
n_layer= 6
dropout= 0.2

In [28]:
torch.manual_seed(1337)

In [29]:
import requests

# Fetch the content from the URL
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
response = requests.get(url)
text = response.text

with open('input.txt', 'w', encoding='utf-8') as f:
    f.write(text)

print(text[:1000]) # prints 1st 1000 chars

# Unique Characters in the text
chars=sorted(list(set(text))) # sorting the characters
vocab_size=len(chars) # number of chars
print(''.join(chars)) # Print the unique characters
print(vocab_size) # Print the number of unique chars

# Chars to Integer Inputs--> Tokenize
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}

# Encoder String to Int
encode=lambda s: [stoi[c] for c in s]

# Decoder Int to String
decode=lambda l: ''.join([itos[i]for i in l])

# Test of encode/decode
print('---------------')
print(encode('PyTorch'))
print(decode(encode('PyTorch')))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [30]:
print("length of the dataset in characters:", len(text))

length of the dataset in characters: 1115394


In [31]:
# 1st 1000 characters above as tokens

data=torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print('---------------')
print(data[:1000])

torch.Size([1115394]) torch.int64
---------------
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39

In [32]:
# Make Train/Test

data=torch.tensor(encode(text), dtype=torch.long)
n=int(0.9*len(data))
train_data=data[:n] # 90% Train
val_data=data[n:] # 10 % Test

In [33]:
#Loads Data

def get_batch(split):
  data=train_data if split =='train' else val_data #define data--> train else test
  ix=torch.randint(len(data)- block_size, (batch_size,)) # random offsets of batch size 32
  x= torch.stack([data[i:i+block_size] for i in ix]) # 1st block size chars
  y= torch.stack([data[i+1:i+block_size+1]for i in ix]) # Offset of x by 1
  return x,y

In [34]:
@ torch.no_grad()
def estimate_loss():
  out={}
  model.eval()
  for split in ['train','val']:
      losses=torch.zeros(eval_iterations)
      for k in range (eval_iterations):
          X,Y = get_batch(split)
          logits, loss = model(X,Y)
          losses[k]=loss.item()
      out[split]=losses.mean
  model.train()
  return out

In [35]:
x=train_data[:block_size] # inputs to transformer
y=train_data[1:block_size+1] # next block size chars
for t in range(block_size): # iterate over block size of 10
  context=x[:t+1]
  target=y [t]
  print(f"When input is {context}, the target is:{target}")

When input is tensor([18]), the target is:47
When input is tensor([18, 47]), the target is:56
When input is tensor([18, 47, 56]), the target is:57
When input is tensor([18, 47, 56, 57]), the target is:58
When input is tensor([18, 47, 56, 57, 58]), the target is:1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is:15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is:47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is:58


In [36]:


def get_batch(split):
  data=train_data if split =='train' else val_data #define data--> train else test
  ix=torch.randint(len(data)- block_size, (batch_size,)) # random offsets of batch size 5
  x= torch.stack([data[i:i+block_size] for i in ix])
  y= torch.stack([data[i+1:i+block_size+1]for i in ix])
  return x,y

xb,yb =get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target=   yb[b,t]
    print(f"When input is{context.tolist()} the target:{target}")

inputs:
torch.Size([32, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
      

In [38]:
print(xb) #input to transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
        [39, 49, 12,  1, 27,  1, 5

In [39]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):

    logits=self.token_embedding_table(idx) #(B,T,C)--> Batch = 5, Time = 10, Channel = vocab_size = 65
    if targets is None:
      loss=None
    else:
      B, T, C= logits.shape
      logits=logits.view(B*T, C) #changes shape of tensor for PyTorch
      targets=targets.view(B*T)
      loss = F.cross_entropy(logits, targets) # loss = cross entropy on logits and targets

    return logits,loss

  def generate(self, idx, max_new_tokens):  #idx is (B,T) array of indices in the current context--> Creates B, T--> B, T+1--> B, T+2--> to max new tokens
    for _ in range(max_new_tokens):
      logits,loss = self(idx) # get the predictions
      logits=logits[:,-1,:]  # focus only on the last time step
      probs=F.softmax(logits, dim=1)  # apply softmax to get probabilities
      idx_next=torch.multinomial(probs, num_samples=1) #sample from the distribution
      idx= torch.cat((idx, idx_next), dim=1) #append sampled index to the running sequence
    return idx




m= BigramLanguageModel(vocab_size)
out=m(xb,yb)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))



torch.Size([256, 65])
tensor(4.7313, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [40]:
logits

tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-0.8109,  0.2410, -0.1139,  ...,  1.4509,  0.1836,  0.3064],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.7470, -1.4852,  0.1714,  ...,  2.2019,  2.4498,  0.6347]],
       grad_fn=<ViewBackward0>)

In [41]:
# Now want to train the model so will make optimizer
optimizer=torch.optim.AdamW(m.parameters(), lr=1e-3) # Used AdamW as optimizer--->


In [43]:
batch_size=32
for steps in range(1000):
  xb,yb= get_batch('train') # sample a batch of data
  logits,loss= m(xb, yb)
  optimizer.zero_grad(set_to_none=True) # Zero out gradients from previous step
  loss.backward() # get gradients for parameters
  optimizer.step() # use gradients to update the parameters

print(loss.item())



3.1285109519958496


In [44]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))



YseL!rS&AD?Rllld fi?PP'dfjZombrs.
tiy caoomptiyk,
FlxjbabeYWI.
Jd
u.PHALEITXx.D?EamIo-Wy!Bawon,;AM 


In [45]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=400)[0].tolist()))


LSqHonQprgmUQnk,lI' ve'g pXcLb bRGjpt!w:C&xrJhyQNCVa!GX

POKNjuvicQ!URT;ApUDriPlyb uGFaQ!UivMkpWx;yFomuBagRIN!pandGRCHhs w.R:VOMuHoi-K oteWAUbQRqp w MURKIDMA:3mpsrv m wMy, CIAwnLEXFlitingh mu--b:CU IGRinQhPUNN&y,cOw,
P sia!ALLw,xcm:&y bt.l&SnkeafrUEHOM&y
QT::-tOthd.
Rrj,
pre?pD.
Whi-zFXXa:,SSBy;VvNNXEHn kyWCHn'da;aIQz,by,
FopxjlXI'z.Ls
HeMpgX&X:thSNengOdmyees isBDYZ3SirlyKuscd
Obe'a!yeDHBUh$whenpo


In [46]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs) Key is Information that token contains
        q = self.query(x) # (B,T,hs) Query is Information token is looking for

        # compute attention scores ("affinities")-->Easiest way for tokens to communicate w/each other is avg of preceeding
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) Upper triangular masking 5th node does not comm with 6, 7,..n
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [47]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [48]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [49]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) #Layer Normalization
        self.ln2 = nn.LayerNorm(n_embd) #Layer Normalization

    def forward(self, x): # These are the residual connections
        x = x + self.sa(self.ln1(x)) # Communication
        x = x + self.ffwd(self.ln2(x)) # Computation
        return x


In [50]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [51]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.693697 M parameters


In [52]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [53]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")



    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

KeyboardInterrupt: ignored